# Parsing a data2dome feed to find and acquire fulldome images

This notebook shows how to connect to a [data2dome](https://docs.google.com/document/d/1USG1clnxSyGf9lsDe-alb6nZzxn_xFzxbjCnSVPzlWU/edit#) JSON feed.

In this example we query the ESO images feed and search for recent images in fulldome format. We then construct an html table with thumbnail images so that we can select which ones we want. Finally we download the desired images, resizing as necessary.

Mark SubbaRao, April 25th, 2017

In [ ]:
import pandas as pd
from IPython.display import Image, HTML
from datetime import date, timedelta
import requests
from PIL import Image as Im
import os

# D2D Feed
ESOimages = 'http://www.eso.org/public/images/d2d/'
howRecent = 180 # in days
pd.set_option('max_colwidth',5000) # so we will return the full description without truncation
numReturn=10 # maximum number of results to return
desiredResolution = 4096

## Parse the Feed

This code block downloads the pagenated json feed and parses it selecting only the images in fulldome format until numReturn objects are found.

In [ ]:
page = ESOimages+'?after='+(date.today()-timedelta(days=howRecent)).strftime('%Y%m%d')+'0000' #the 0000 is the time component of the date
nImg = 0
nPage=0
feedImages=[]
while (page!='' and nImg < numReturn):
    print(page)
    df=pd.read_json(page)
    # The Next parameter may or may not exist
    try:
        page=df.Next[0]
    except:
        page=''
    nPage=nPage+1
    for feedImage in df.Collections:
        if (feedImage['Assets'][0]['Resources'][0]['ProjectionType']=='Fulldome'and nImg < numReturn):
            feedImages.append(feedImage)
            nImg=nImg+1
    print(str(nPage) + ' pages parsed ' + str(nImg) + ' fulldome images found')

## Create table with thumbnail images for image selection

In [ ]:
titleList=[]
descriptionList=[]
thumbnailList=[]
pubdateList=[]
for pic in feedImages:
    titleList.append(pic['Title'])
    descriptionList.append(pic['Description'].replace("\r\n",''))
    pubdateList.append(pic['PublicationDate'])
    for resource in pic['Assets'][0]['Resources']:
        if (resource['ResourceType']=='Thumbnail'):
            thumbnailList.append('<img src="'+resource['URL']+'"/>')            

df3 = pd.DataFrame({'title':titleList,'description':descriptionList,'thumbnail':thumbnailList,'pubdate':pubdateList},columns=['title','thumbnail','description','pubdate'])

HTML(df3.to_html(escape=False))

## Selecting and downloading the images

The next code block contains a function to download the image, resizing it if it is larger than the desired size (note some of the ESO fisheye images are very large (~16K). 

In [ ]:
def getD2Dimage(num,imageName):
    imURL=''
    imSize=0
    imDict = feedImages[num]
    
    # Download image, resize if necessary and save in module directory
    for resource in imDict['Assets'][0]['Resources']:
        if ((resource['Dimensions'][0]>=imSize and imSize < desiredResolution) or (resource['Dimensions'][0]<=imSize and imSize > desiredResolution and resource['Dimensions'][0] > desiredResolution)):
            imSize = resource['Dimensions'][0]
            imURL = resource['URL']
    print('downloading '+imURL)    
    im = Im.open(requests.get(imURL, stream=True).raw)
    if (imSize > desiredResolution):
        im = im.resize((desiredResolution,desiredResolution))
        print('resizing image from '+str(imSize)+' to '+str(desiredResolution))
    im.save(imageName)

Here is an example of how to call the getD2Dimage function

In [ ]:
getD2Dimage(0,'LaSilla.jpg')